In [16]:
import openai

client = openai.OpenAI()
messages = []

In [17]:
def call_ai():
     response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
     )
     message = response.choices[0].message.content
     messages.append({"role": "assistant", "content": message})
     print(f"Ai: {message}")

In [18]:
while True:
    message = input("Send a message to the LLM...")
    if message == 'quit' or message == 'q':
        break
    else :
        messages.append({"role": "user", "content": message})
        print(f"user: {message}")
        call_ai()

user: 내 이름은 기원이야
Ai: 안녕하세요, 기원님! 어떻게 도와드릴까요?
user: 내 이름은 뭐야?
Ai: 당신의 이름은 기원이세요. 맞나요?
user: 나는 한국 출신이야
Ai: 그렇군요! 한국은 아름다운 문화와 역사를 가진 나라죠. 한국에 대해 이야기해보고 싶은 주제가 있나요?
user: 내가 물어본 첫번째 질문과 내가 태어난 나라와 가장 가까운 섬이 어디야?
Ai: 당신이 한국 출신이라면, 한국에서 가장 가까운 섬 중 하나는 **제주도**입니다. 제주도가 한국 본토에서 떨어져 있지만, 한국에서 가장 큰 섬으로 유명하죠. 다른 질문이 있으신가요?
